In [0]:
import sys
import os

# 1. Configura Path para importar 'src'
repo_root = os.getcwd()
repo_root = '/'.join(repo_root.split('/')[:-1])
if repo_root not in sys.path:
    sys.path.append(repo_root)



In [0]:
%pip install -r ../requirements.txt

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from src.utility.environment import Environment
from src.infrastructure.data_manager import DataManager
from src.infrastructure.ml_trainer import PySparkTrainer

In [0]:
# 1. Carregar Dados
manager = DataManager(spark)
df_gold = manager.read_delta(f"{Environment.feature_store_path}/churn")

In [0]:
df_gold.head(5)

In [0]:
df_gold.describe().show()

In [0]:
df_gold.pandas_api()['churn'].value_counts(normalize=True) * 100

Desbalanceamento elevado. Necessário fazer downsampling, usar peso para as classes ou outras estratégias para lidar com o problema

In [0]:
num_cols = [
    'recency', 'frequency', 'monetary',
    'total_items_volume', 'max_installments', 'avg_satisfaction'
]

for coll in num_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(df_gold.toPandas()[coll], kde=True)
    plt.title(f"Distribution of {coll}")
    plt.show()

- Frequencia bem desbalanceado
- Satisfação consideravelmente positiva
- Item e monetary bastante destorcido para valores mais baixos

## Analise das categorias

In [0]:
cat_cols = ['state', 'primary_payment_type']

for coll in cat_cols:
    plt.figure(figsize=(8,4))
    df_gold.toPandas()[coll].value_counts().head(10).plot(kind='bar')
    plt.title(f"Top categories of {col}")
    plt.show()

## Interpretação
- Poucos estados dominam as vendas
- Majoritariamente o tipo de venda é por cartão de crédito, embora boleto possua relevância

In [0]:
plt.figure(figsize=(12,8))
sns.heatmap(
    df_gold.select([col("recency"), col("frequency"), col("monetary"), col('total_items_volume'), 
                    col('payment_method_count'), col('max_installments'), 
                    col('avg_satisfaction'), col('min_review_score'), col("total_reviews_given")]).pandas_api().corr().to_pandas(),
    cmap='coolwarm',
    annot=True
)
plt.title("Correlation Matrix")
plt.show()

## Correlações mais fortes
- Frequencia - Total de Reviews -> Bem forte
- Frequencia - total_items
- Total Items - Total Reviews
- Monetary - Max installments
- Monetary - Total_items

# Entendimentos
## Pontos Chave
- Majoritariamente os compradores fazem apenas uma compra
- Desbalanceamento de classes bastante significativo - ~98% vs 2%
- Variaveis indepentes
## Pontos de Atenção
- Utilizar peso nos modelos
- Otimizar para Recall, F1 ou ROC AUC
- Não utilizar acurácia